# Keras Model

Before you begin using Keras, import the required modules. If you are having problems with this step, make sure you have installed all dependencies found in requirements.txt and have selected the correct Python kernel (in the top menu, "Kernel"->"Change kernel"). If you are using virtualenv, your kernel will not show up unless you follow the instructions found [here](https://stackoverflow.com/questions/37891550/jupyter-notebook-running-kernel-in-different-env).

In [42]:
import numpy as np
import pandas as pd

import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense
from keras.optimizers import SGD

## Data Preprocessing

First, we want to grab the dataset from the CSV file. Load it as a Pandas Dataframe so we can easily work with it in further steps.

In [56]:
def get_dataset(path):
    dataset = pd.read_csv(path)
    np.random.seed(42)
    dataset = dataset.reindex(np.random.permutation(dataset.index))
    return dataset

dataset = get_dataset('data/dataset.csv')
dataset.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
204,87930,B,12.47,18.60,81.09,481.9,0.09965,0.1058,0.08005,0.03821,...,24.64,96.05,677.9,0.1426,0.2378,0.2671,0.10150,0.3014,0.08750,NaN
70,859575,M,18.94,21.31,123.60,1130.0,0.09009,0.1029,0.10800,0.07951,...,26.58,165.90,1866.0,0.1193,0.2336,0.2687,0.17890,0.2551,0.06589,NaN
131,8670,M,15.46,19.48,101.70,748.9,0.10920,0.1223,0.14660,0.08087,...,26.00,124.90,1156.0,0.1546,0.2394,0.3791,0.15140,0.2837,0.08019,NaN
431,907915,B,12.40,17.68,81.47,467.8,0.10540,0.1316,0.07741,0.02799,...,22.91,89.61,515.8,0.1450,0.2629,0.2403,0.07370,0.2556,0.09359,NaN
540,921385,B,11.54,14.44,74.65,402.9,0.09984,0.1120,0.06737,0.02594,...,19.68,78.78,457.8,0.1345,0.2118,0.1797,0.06918,0.2329,0.08134,NaN


Now, we'll write functions to get the data and the labels (i.e. the x and the y). We want to make sure these return NumPy arrays so they can be passed into Keras models.

In [32]:
def get_data(dataset):
    data = dataset.as_matrix()[:, 2:-1]
    return data

def get_labels(dataset):
    labels = dataset['diagnosis'].replace(['M', 'B'], [1, 0]).as_matrix()
    return labels

data, labels = get_data(dataset), get_labels(dataset)
print(data.shape)

(569, 30)


Finally, we'll split the data and labels into train, validation, and test sets.

In [37]:
def split_data(data, labels, split):
    train_ratio, val_ratio, test_ratio = split
    num_examples = labels.shape[0]
    train_bound, val_bound = int(train_ratio*num_examples), int(train_ratio*num_examples) + int(val_ratio*num_examples)
    
    train = {'data': data[:train_bound], 'labels': labels[:train_bound]}
    val = {'data': data[train_bound:val_bound], 'labels': labels[train_bound:val_bound]}
    test = {'data': data[val_bound:], 'labels': labels[val_bound:]}
    
    return train, val, test
    
train, val, test = split_data(data, labels, (.7, .2, .1))

## Model Architecture and Training

Let's define our model's architecture first. We want a 4-layer Fully-Connected Network that can be used for binary classification.

In [38]:
def create_model():
    model = Sequential()
    
    model.add(Dense(16, activation='relu', input_dim=30))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    sgd = SGD(lr=0.01)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['binary_accuracy'])
    
    return model

model = create_model()

Now let's train our model! We want to not only train the model with many epochs, but also print the validation set accuracy at every epoch.

In [50]:
def train_model(model, train, val, num_epochs):
    for _ in range(num_epochs):
        model.fit(train['data'], train['labels'], epochs=1, verbose=1, batch_size=16)
        score = model.evaluate(val['data'], val['labels'], batch_size=16, verbose=0)
        print("Validation accuracy: %.2f\n" % (score[1]))
        
train_model(model, train, val, 10)

test_score = model.evaluate(test['data'], test['labels'], batch_size=128)
print("Test accuracy: %.2f" % (test_score[1]))

Epoch 1/1
398/398 [==============================] - 0s - loss: 7.0061 - binary_accuracy: 0.5653     
Validation accuracy: 0.78

Epoch 1/1
398/398 [==============================] - 0s - loss: 7.0061 - binary_accuracy: 0.5653     
Validation accuracy: 0.78

Epoch 1/1
398/398 [==============================] - 0s - loss: 7.0061 - binary_accuracy: 0.5653     
Validation accuracy: 0.78

Epoch 1/1
398/398 [==============================] - 0s - loss: 7.0061 - binary_accuracy: 0.5653     
Validation accuracy: 0.78

Epoch 1/1
398/398 [==============================] - 0s - loss: 7.0061 - binary_accuracy: 0.5653     
Validation accuracy: 0.78

Epoch 1/1
398/398 [==============================] - 0s - loss: 7.0061 - binary_accuracy: 0.5653     
Validation accuracy: 0.78

Epoch 1/1
398/398 [==============================] - 0s - loss: 7.0061 - binary_accuracy: 0.5653     
Validation accuracy: 0.78

Epoch 1/1
398/398 [==============================] - 0s - loss: 7.0061 - binary_accuracy: 0.5653 